In [1]:
import random
import pandas as pd
import json
import ast
import xlsxwriter

In [2]:
import neal
import dimod
from dimod import Binary,Real
import numpy as np
from dimod import ConstrainedQuadraticModel,ExactCQMSolver
from dwave.system import LeapHybridCQMSampler
from itertools import chain
import copy
import xlsxwriter
import random
import copy
import time
from pulp import LpVariable, LpProblem, lpSum, LpMaximize, LpMinimize,LpStatus,listSolvers,getSolver
import warnings
warnings.filterwarnings('ignore')

In [3]:
# create constraints values for sizes, based on different limits
def constraints_func(dl, gs):
    dl_limits, cons = [], []
    c = 0
    dl.reverse()
    for i in dl:
        c = c + i
        dl_limits.append(c)
        if c > gs:
            cons.append(gs)
        else:
            cons.append(c)
    return cons, dl_limits


# measure sum total of priority,size etc  
def total(sizes, result):
    sum_size = []
    for i in result:
        sum_size.append(sizes[i])

    return sum(sum_size)


# data manupilation
def correct(lst):
    for i in range(int(len(lst) / 2)):
        lst = add_negative_pairs(lst)
    return lst


# takes series for targets and gs, returns target list and gs list well seprated with its limits
def separate_gs_and_fix_non_gs_targets(s):
    if s[-1] > 0:
        t = []
        for element in reversed(s):
            if element < 0:
                break
            t.append(element)

        s = t[::-1] + s

    s = correct(s)
    s_, temp, gs_ = [], [], []
    for i in s:
        if i < 0:
            gs_.append(i)
            s_.append(temp)
            temp = []
        else:
            temp.append(i)

    if not s_[0]:
        s_ = s_[1:]
        gs_ = gs_[1:]

    return s_, gs_


def is_equal_or_less(list1, list2):
    return all(x >= y for x, y in zip(list1, list2))


# post processing and mapping the results in quadrents and lower level checks on storage and energy
def post(res, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_, 
         q_targets, sizes, energy_limit_, a_time, result_col, priority_col, time_col, conditions_satisfied_col,
        unsatisfied_conditions_col):
    parsed_results = []
    total_size = 0
    priority_results = 0
    total_energy = 0
    conditions_satisfied = True
    unsatisfied_conditions = []
    for j in res:
        for i in converted_dict.items():
            if j in i[1]:
                parsed_results.append(i[0])
                # print(size_dict[i[0]])
                total_size = total_size + size_dict[i[0]]
                priority_results = priority_results + priorities_dict[i[0]]
                total_energy = total_energy + energies_dict[i[0]]

    i_, isize, iq, ienergy, sum_s, sum_e, sum_size, sum_energy = [], [], [], [], [], [], [], []
    for i in target_:
        j_, jsize, jq, je, um_s, um_e = [], [], [], [], [], []
        for j in i:
            k_, ksize, kq, ke = [], [], [], []
            for k in j:
                if k in res:
                    k_.append(k)
                    kq.append(q_targets[k])
                    ksize.append(sizes[k])
                    ke.append(energy[k])
            j_.append(k_)
            jq.append(kq)
            jsize.append(ksize)
            je.append(ke)
            um_s.append(sum(ksize))
            um_e.append(sum(ke))

        i_.append(j_)
        iq.append(jq)
        isize.append(jsize)
        ienergy.append(je)
        sum_s.append(um_s)
        sum_e.append(um_e)
        sum_size.append(sum(um_s))
        sum_energy.append(sum(um_e))

    print()
    print("result", iq)
    print()
    print("sizes", isize)
    print("sizes", sum_s)
    print("sizes", sum_size)
    
    sheet.write(row, result_col, str(list(chain(*list(chain(*iq))))))


    gs_list = [gs1_, gs2_, gs3_]
    print("Gs 1,2,3 limit:", gs_list)
    for i, j, k in zip(gs_list, sum_s, sum_size):
        print("Sum of ground stations:", sum(i))
        if k > (-1 * sum(i)):
            print("Storage total is beyond", k, -1 * sum(i))
        if j[-1] > (-1 * i[-1]):
            print("Storage not satisfied", j[-1], -1 * i[-1])
            conditions_satisfied = False
            if "Storage limit not satisfied" not in unsatisfied_conditions:
                unsatisfied_conditions.append("Storage limit not satisfied")

    print("energy", ienergy)
    print("energy", sum_e)
    print("energy", sum_energy)

    if not is_equal_or_less(energy_limit_, sum_energy):
        print("Energy not satisfied")
        conditions_satisfied = False
        if "Energy limit not satisfied" not in unsatisfied_conditions:
            unsatisfied_conditions.append("Energy limit not satisfied")

    print()
    print("Total priority:", priority_results)
    sheet.write(row, priority_col, int(priority_results))
    print(f"exceution time :{a_time} ms")
    sheet.write(row, time_col, str(a_time) + " ms")
    sheet.write(row, conditions_satisfied_col, str(conditions_satisfied))
    sheet.write(row, unsatisfied_conditions_col, str(unsatisfied_conditions))
    


In [4]:
def add_negative_pairs(lst):
    # base case: if the list is empty, return an empty list
    if not lst:
        return []

    # if the first element is positive, keep it and recurse on the rest of the list
    if lst[0] >= 0:
        return [lst[0]] + add_negative_pairs(lst[1:])

    # if the first two elements are negative, add them and recurse on the rest of the list
    if len(lst) > 1 and lst[1] < 0:
        return [lst[0] + lst[1]] + add_negative_pairs(lst[2:])

    # if the first element is negative and the second is positive, keep the negative element and recurse on the rest of the list
    return [lst[0]] + add_negative_pairs(lst[1:])

In [5]:
def convert_gs_to_gs_limit(satellite, ground_stations, gs_limit):
    satellite_coverage = []
    for sat in satellite:
        if isinstance(sat, int):
            satellite_coverage.append(sat)
        else:
            gs_index = ground_stations.index(int(sat.replace('GS', '')))               
            satellite_coverage.append(gs_limit[gs_index])
    return satellite_coverage

In [6]:
excel_file = 'data/data-eos.xlsx'
eos_data_df = pd.read_excel(excel_file)

In [7]:
eos_data_df.head()

,Targets,Priorities,Size of Targets,Energies of Targets,Storage Limit of Satellites,Energy Limit of Satellites,Ground Stations,Limit of Ground Stations,Satellite 1,Satellite 2,Satellite 3
0,"['T626', 'T981', 'T636', 'T445', 'T36', 'T23',...","[370, 46, 599, 553, 863, 864, 24, 963, 38, 908...","[49, 261, 213, 941, 472, 639, 789, 807, 524, 3...","[349, 191, 336, 42, 515, 712, 779, 541, 930, 9...","[2946, 3229, 2901]","[1810, 2663, 1788]","['GS102', 'GS172', 'GS78', 'GS61', 'GS26', 'GS...","[363, 300, 322, 393, 293, 300, 334, 311, 313, ...","[451, 74, 770, 276, 226, 358, 512, 617, 441, 2...","[266, 'GS143', 235, 70, 142, 253, 816, 732, 93...","[877, 531, 318, 638, 315, 'GS172', 552, 111, 4..."
1,"['T615', 'T192', 'T925', 'T904', 'T485', 'T907...","[244, 660, 690, 349, 258, 966, 584, 312, 329, ...","[938, 466, 872, 641, 673, 191, 367, 159, 760, ...","[903, 830, 994, 232, 141, 659, 564, 617, 482, ...","[2735, 3097, 2844]","[2139, 1025, 1428]","['GS34', 'GS44', 'GS167', 'GS125', 'GS139', 'G...","[325, 311, 335, 328, 367, 369, 340, 297, 308, ...","[504, 520, 367, 'GS59', 928, 897, 329, 'GS83',...","[732, 750, 170, 'GS68', 'GS4', 'GS42', 735, 44...","[893, 'GS173', 'GS14', 859, 365, 606, 'GS27', ..."
2,"['T129', 'T677', 'T799', 'T53', 'T950', 'T212'...","[936, 351, 785, 467, 523, 754, 630, 192, 665, ...","[974, 920, 869, 176, 59, 455, 651, 483, 456, 9...","[478, 845, 379, 502, 847, 524, 411, 997, 90, 1...","[2971, 3417, 3217]","[1834, 2637, 2030]","['GS150', 'GS98', 'GS35', 'GS134', 'GS111', 'G...","[330, 283, 282, 382, 345, 301, 382, 377, 273, ...","['GS13', 387, 137, 529, 884, 483, 248, 'GS100'...","['GS35', 'GS173', 72, 'GS116', 'GS93', 'GS43',...","['GS26', 'GS29', 'GS91', 'GS109', 'GS68', 'GS5..."
3,"['T963', 'T537', 'T512', 'T899', 'T146', 'T703...","[461, 499, 423, 50, 818, 773, 991, 310, 386, 8...","[315, 953, 242, 293, 273, 745, 255, 738, 478, ...","[749, 930, 11, 460, 644, 736, 965, 854, 596, 6...","[3477, 3986, 2904]","[2103, 1391, 2557]","['GS169', 'GS24', 'GS39', 'GS172', 'GS92']","[282, 281, 346, 348, 319]","[257, 836, 964, 286, 973, 690, 377, 789, 576, ...","['GS172', 472, 988, 688, 958, 709, 667, 91, 36...","[576, 5, 946, 581, 836, 77, 'GS24', 798, 846, ..."
4,"['T345', 'T66', 'T542', 'T594', 'T408', 'T908'...","[868, 134, 343, 39, 482, 732, 217, 284, 698, 2...","[839, 123, 779, 777, 276, 498, 840, 203, 293, ...","[880, 488, 566, 192, 723, 978, 130, 242, 421, ...","[3806, 2706, 3003]","[1142, 1436, 1696]","['GS148', 'GS70', 'GS105', 'GS137', 'GS42', 'G...","[285, 294, 342, 361, 372, 298, 338, 310, 272, ...","[537, 'GS60', 384, 'GS155', 'GS9', 161, 999, '...","[173, 811, 948, 883, 685, 519, 188, 731, 627, ...","[231, 435, 'GS154', 627, 758, 940, 595, 593, 8..."


In [8]:
input_json_array = []
for index, row in eos_data_df.iterrows():
    input_json_array.append({'Targets': row['Targets'], 'Priorities': row['Priorities'],
                             'Size of Targets': row['Size of Targets'], 
                             'Energies of Targets': row['Energies of Targets'],
                             'Storage Limit of Satellites': row['Storage Limit of Satellites'], 
                             'Energy Limit of Satellites': row['Energy Limit of Satellites'],
                             'Ground Stations': row['Ground Stations'], 
                             'Limit of Ground Stations': row['Limit of Ground Stations'],
                             'Satellite 1': row['Satellite 1'], 'Satellite 2': row['Satellite 2'],
                             'Satellite 3': row['Satellite 3']                            
                            })
input_json = json.dumps(input_json_array)

In [9]:
book = xlsxwriter.Workbook('data/results-eos.xlsx')
sheet = book.add_worksheet('sheet1')

content = ["Quantum Result", "Total Priority", "Execution Time", "Satisified all Constraints?",
           "UnSatisfied Constraints", "Simulated Annealing Result", "Total Priority", "Execution Time", 
           "Satisified all Constraints?","Constraints Not Satisfied", 
           "Gurobi Optimizer Result", "Total Priority", "Execution Time", "Satisified all Constraints?",
           "Constraints Not Satisfied"]

sheet.write(0, 0, content[0])
sheet.write(0, 1, content[1])
sheet.write(0, 2, content[2])
sheet.write(0, 3, content[3])
sheet.write(0, 4, content[4])
sheet.write(0, 5, content[5])
sheet.write(0, 6, content[6])
sheet.write(0, 7, content[7])
sheet.write(0, 8, content[8])
sheet.write(0, 9, content[9])
sheet.write(0, 10, content[10])
sheet.write(0, 11, content[11])
sheet.write(0, 12, content[12])
sheet.write(0, 13, content[13])
sheet.write(0, 14, content[14])

0

In [11]:
output = json.loads(input_json)
row = 1;
for out in output:
    targets = [int(target.replace('T', '')) for target in ast.literal_eval(out['Targets'])]
    priorities = [priority for priority in ast.literal_eval(out['Priorities'])]
    sizes = [priority for priority in ast.literal_eval(out['Size of Targets'])]
    energies = [energy for energy in ast.literal_eval(out['Energies of Targets'])]
    ground_stations = [int(gs.replace('GS', '')) for gs in ast.literal_eval(out['Ground Stations'])]
    gs_limit = [-gs_limit for gs_limit in ast.literal_eval(out['Limit of Ground Stations'])]
    s1 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(out['Satellite 1'])],
                                ground_stations, gs_limit)
    s2 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(out['Satellite 2'])],
                                ground_stations, gs_limit)
    s3 = convert_gs_to_gs_limit([s1 for s1 in ast.literal_eval(out['Satellite 3'])],
                                ground_stations, gs_limit)
    energy_limit = [energy_limit for energy_limit in ast.literal_eval(out['Energy Limit of Satellites'])]
    storage_limit = [storage_limit for storage_limit in ast.literal_eval(out['Storage Limit of Satellites'])]

    s1_, gs1_ = separate_gs_and_fix_non_gs_targets(s1)

    s2_, gs2_ = separate_gs_and_fix_non_gs_targets(s2)

    s3_, gs3_ = separate_gs_and_fix_non_gs_targets(s3)

    size_dict = dict(zip(targets, sizes))

    priorities_dict = dict(zip(targets, priorities))

    energies_dict = dict(zip(targets, energies))
    

    size1_list = [[size_dict[key] for key in sublist] for sublist in s1_]
    # print("size1_list",size1_list)

    size2_list = [[size_dict[key] for key in sublist] for sublist in s2_]
    # print("size2_list",size2_list)

    size3_list = [[size_dict[key] for key in sublist] for sublist in s3_]
    # print("size3_list",size3_list)

    priorities1_list = [[priorities_dict[key] for key in sublist] for sublist in s1_]
    # print("priorities1_list",priorities1_list)

    priorities2_list = [[priorities_dict[key] for key in sublist] for sublist in s2_]
    # print("priorities2_list",priorities2_list)

    priorities3_list = [[priorities_dict[key] for key in sublist] for sublist in s3_]
    # print("priorities3_list",priorities3_list)

    energies1_list = [[energies_dict[key] for key in sublist] for sublist in s1_]
    # print("energies1_list",energies1_list)

    energies2_list = [[energies_dict[key] for key in sublist] for sublist in s2_]
    # print("energies2_list",energies2_list)

    energies3_list = [[energies_dict[key] for key in sublist] for sublist in s3_]
    # print("energies3_list",energies3_list)

    print()

    intial_target = [s1_, s2_, s3_]
    # print("Targets = ",intial_target)

    target_dict = {}
    c = 0
    target_ = []
    for i in intial_target:
        j_ = []
        for j in i:
            k_ = []
            for k in j:
                k_.append(c)
                target_dict[c] = k
                c = c + 1
            j_.append(k_)
        target_.append(j_)

        # print("Target_suitable_for_Quantum = ",target_)
    print()
    # print("Target_mapping = ",target_dict)

    size_ = [size1_list, size2_list, size3_list]
    priority_ = [priorities1_list, priorities2_list, priorities3_list]
    energy_ = [energies1_list, energies2_list, energies3_list]

    energy_limit_ = energy_limit
    # energy limit for each satellite
    assert len(energy_) == len(energy_limit_)

    dl_ = [gs1_, gs2_, gs3_]
    dl_copy = copy.deepcopy(dl_)
    # downlink size limit in satellites 

    assert len(dl_) == len(target_)
    dl_sizes = [len(i) for i in dl_]
    quadrent_sizes = [len(i) for i in target_]
    assert dl_sizes == quadrent_sizes

    gs_ = storage_limit
    # satellite storge in satellites

    assert len(gs_) == len(target_)

    name = 0

    q_targets = list(chain(*list(chain(*intial_target))))
    targets = list(chain(*list(chain(*target_))))
    sizes = list(chain(*list(chain(*size_))))
    priority = list(chain(*list(chain(*priority_))))
    energy = list(chain(*list(chain(*energy_))))

    assert len(targets) == len(sizes)
    assert len(priority) == len(targets)
    assert len(energy) == len(sizes)


    dl_ = [[-1 * j for j in i] for i in dl_]

    converted_dict = {}
    for key, value in target_dict.items():
        if value in converted_dict:
            converted_dict[value].append(key)
        else:
            converted_dict[value] = [key]

    keys = [k for k, v in converted_dict.items() if len(v) > 1]

    options_ = []
    for key in keys:
        options_.append(converted_dict[key])

    pstart = time.time()
    cqm = ConstrainedQuadraticModel()
    targets_included = []
    for j in range(len(priority)):
        targets_included.append(-priority[j] * Binary(f't{targets[j]}'))
    cqm.set_objective(sum(targets_included))
    target_result = [(k) * Binary(f't{j}') for j, k in zip(range(len(targets)), sizes)]

    energy_array = [(k) * Binary(f't{j}') for j, k in zip(range(len(targets)), energy)]

    for target, size, dl, gs, energy_limit in zip(target_, size_, dl_, gs_, energy_limit_):
        count = target[0][0]
        cons, dl_limits = constraints_func(dl, gs)
        cons.reverse()
        counts = []
        for i in zip(target, cons):
            cqm.add_constraint(sum(target_result[count:count + len(i[0])]) <= i[1],
                               label=f'per q {name} {count}:{count + len(i[0])}<={i[1]}')
            name = name + 1
            counts.append(count)
            count = count + len(i[0])

        dl_limits.reverse()

        for i in zip(counts[:-1], dl_limits[:-1]):
            cqm.add_constraint(sum(target_result[i[0]:count]) <= i[1], label=f'per q {name} {i[0]}:{count}<={i[1]}')
            name = name + 1

        cqm.add_constraint(sum(energy_array[target[0][0]:count]) <= energy_limit,
                           label=f'{name} energy limit {target[0][0]}:{count} {energy_limit}')

    for i in range(len(options_)):
        cqm.add_constraint(sum([Binary(f't{j}') for j in options_[i]]) <= 1, label=f'common {i} {options_[i]}')

    pend = time.time()

    preprocessing_time = (pend - pstart) * 10 ** 3

    qstart = time.time()

    sampler = LeapHybridCQMSampler(token="DEV-ffd3b14ef174e6777d0b3316e9de3be1fa64c470")
    sampleset = sampler.sample_cqm(cqm)
    # sampleset = ExactCQMSolver().sample_cqm(cqm)
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
    result = []
    for i in feasible_sampleset.first.sample.items():
        if i[1] == 1.0:
            result.append(int(i[0].replace("t", "")))
    result.sort()
    print("result = ", result)
    sheet.write(row, 0, str(result))

    qend = time.time()
    q_time = (qend - qstart) * 10 ** 3

    post(result, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_, q_targets, sizes,
         energy_limit_, q_time, 0, 1, 2, 3, 4)

    cstart = time.time()

    bqm, invert = dimod.cqm_to_bqm(cqm)
    qubo = bqm.to_qubo()
    sampler = neal.SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, seed=1234,
                               beta_range=[0.1, 4.2],
                               num_sweeps=20,
                               beta_schedule_type='geometric')
    data = sampleset.first.sample

    sa = []
    for i in targets:
        if (data[f"t{i}"]) == 1:
            sa.append(i)

    cend = time.time()
    c_time = (cend - cstart) * 10 ** 3

    print()
    print("Simulated Annealing results")
    post(sa, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_, q_targets, sizes,
         energy_limit_, c_time, 5, 6, 7, 8, 9)

    pend = time.time()

    # Define the optimization problem
    problem = LpProblem("Target Selection", LpMinimize)

    # Define binary variables for each target
    targets_vars = {f't{j}': LpVariable(f't{j}', cat='Binary') for j in range(len(targets))}

    # Define the objective function
    problem += lpSum(-priority[j] * targets_vars[f't{j}'] for j in range(len(priority)))

    # Define the target_result and energy_array
    target_result = [sizes[j] * targets_vars[f't{j}'] for j in range(len(targets))]
    energy_array = [energy[j] * targets_vars[f't{j}'] for j in range(len(targets))]

    # Define constraints
    for target, size, dl, gs, energy_limit in zip(target_, size_, dl_, gs_, energy_limit_):
        count = target[0][0]
        cons, dl_limits = constraints_func(dl, gs)
        cons.reverse()
        counts = []
        for i in zip(target, cons):
            problem += lpSum(target_result[count:count + len(i[0])]) <= i[
                1], f'per q {name} {count}:{count + len(i[0])}<={i[1]}'
            name = name + 1
            counts.append(count)
            count = count + len(i[0])

        dl_limits.reverse()

        for i in zip(counts[:-1], dl_limits[:-1]):
            problem += lpSum(target_result[i[0]:count]) <= i[1], f'per q {name} {i[0]}:{count}<={i[1]}'
            name = name + 1

        problem += lpSum(energy_array[target[0][
                                          0]:count]) <= energy_limit, f'{name} energy limit {target[0][0]}:{count} {energy_limit}'

    for i in range(len(options_)):
        problem += lpSum(targets_vars[f't{j}'] for j in options_[i]) <= 1, f'common {i} {options_[i]}'

    print()
    print("Pulp results")
    solver = getSolver('GUROBI')
    solver.solve(problem)
    sol = []
    # Print the solution
    # print("Status:", LpStatus[problem.status])
    for v in problem.variables():
        if v.varValue == 1.0:
            sol.append(int(str(v.name).split("t")[-1]))


    pend = time.time()

    p_time = (pend - pstart) * 10 ** 3
    # print(sol)
    post(sol, converted_dict, size_dict, priorities_dict, energies_dict, target_, gs1_, gs2_, gs3_, q_targets, sizes,
         energy_limit_, p_time, 10, 11, 12, 13, 14)

    print("...........................................................................................................")
    print()
    print()
    row = row + 1

book.close()




Simulated Annealing results

result [[[93, 697, 310], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [771, 735, 212]], [[], [], [], [755], [775], [168, 463], [999, 512, 711], [], [], [], [], [], [], [], [], [115, 743]], [[], [], [], [], [], [], [], [], [524, 803, 280, 152], [231], [], [], [], [], [], [], [], []]]

sizes [[[724, 783, 546], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [187, 209, 337]], [[], [], [], [283], [173], [191, 190], [116, 124, 177], [], [], [], [], [], [], [], [], [571, 507]], [[], [], [], [], [], [], [], [], [912, 992, 974, 630], [596], [], [], [], [], [], [], [], []]]
sizes [[2053, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 733], [0, 0, 0, 283, 173, 381, 417, 0, 0, 0, 0, 0, 0, 0, 0, 1078], [0, 0, 0, 0, 0, 0, 0, 0, 3508, 596, 0, 0, 0, 0, 0, 0, 0, 0]]
sizes [2786, 2332, 4104]
******** [93, 697, 310, 771, 735, 212, 755, 775, 168, 463, 999, 512, 711, 115, 743, 524, 803, 280, 152, 231]
Gs 1,2,3 limit: [[-357, -363, -311, -




Simulated Annealing results

result [[[], [], [], [], [124], [151], [531, 711], [], [], [830], [], [], []], [[], [21], [720, 458], [3, 704, 611], [], [589, 391], [592], [], [], [964], [], [806], [], [], [], [533], [], [], [], [], [], [], [], [], [], [552], [790, 212]], [[], [], [], [], [46], [592], [], [], [], [], [], [], [], [], [], [], [276, 739, 900, 999, 336], [991], [], [], [], [], [], [], [], [], []]]

sizes [[[], [], [], [], [733], [383], [847, 501], [], [], [962], [], [], []], [[], [702], [961, 806], [221, 309, 46], [], [162, 165], [175], [], [], [179], [], [75], [], [], [], [157], [], [], [], [], [], [], [], [], [], [945], [756, 738]], [[], [], [], [], [239], [175], [], [], [], [], [], [], [], [], [], [], [905, 600, 492, 810, 703], [848], [], [], [], [], [], [], [], [], []]]
sizes [[0, 0, 0, 0, 733, 383, 1348, 0, 0, 962, 0, 0, 0], [0, 702, 1767, 576, 0, 327, 175, 0, 0, 179, 0, 75, 0, 0, 0, 157, 0, 0, 0, 0, 0, 0, 0, 0, 0, 945, 1494], [0, 0, 0, 0, 239, 175, 0, 0, 0, 0, 0, 0, 